In [43]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats

In [44]:
filename = "data/speeddating.csv"

data = pd.read_csv(filename, sep=',', low_memory=False)

data.head()


,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,...,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,0,1,female,21,27,6,[4-6],'Asian/Pacific Islander/Asian-American',European/Caucasian-American,0,...,[0-3],[3-5],7,6,[6-8],[5-6],0,1,0,0
1,0,1,female,21,22,1,[0-1],'Asian/Pacific Islander/Asian-American',European/Caucasian-American,0,...,[0-3],[3-5],7,5,[6-8],[5-6],1,1,0,0
2,1,1,female,21,22,1,[0-1],'Asian/Pacific Islander/Asian-American','Asian/Pacific Islander/Asian-American',1,...,[0-3],[3-5],7,?,[6-8],[0-4],1,1,1,1
3,0,1,female,21,23,2,[2-3],'Asian/Pacific Islander/Asian-American',European/Caucasian-American,0,...,[0-3],[3-5],7,6,[6-8],[5-6],0,1,1,1
4,0,1,female,21,24,3,[2-3],'Asian/Pacific Islander/Asian-American','Latino/Hispanic American',0,...,[0-3],[3-5],6,6,[6-8],[5-6],0,1,1,1


In [45]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 123 entries, has_null to match
dtypes: int64(7), object(116)
memory usage: 7.9+ MB


In [62]:
# List of columns to delete
cols_to_drop = [
    'wave', 'position', 'position_o', 'pid', 'iid', 'id', 'partner', 'pid_o',
    'round', 'has_null', 'samerace', 'd_age', 'd_field', 'd_race',
    'd_importance_same_race', 'd_importance_same_religion',
    'd_pref_o_attractive', 'd_pref_o_sincere', 'd_pref_o_intelligence',
    'd_pref_o_funny', 'd_pref_o_ambitious', 'd_pref_o_shared_interests',
    'd_attractive_o', 'd_sinsere_o', 'd_intelligence_o', 'd_funny_o',
    'd_ambitous_o', 'd_shared_interests_o', 'd_attractive_important',
    'd_sincere_important', 'd_intellicence_important', 'd_funny_important',
    'd_ambtition_important', 'd_shared_interests_important', 'd_attractive',
    'd_sincere', 'd_intelligence', 'd_funny', 'd_ambition',
    'd_attractive_partner', 'd_sincere_partner', 'd_intelligence_partner',
    'd_funny_partner', 'd_ambition_partner', 'd_shared_interests_partner',
    'd_sports', 'd_tvsports', 'd_exercise', 'd_dining', 'd_museums', 'd_art',
    'd_hiking', 'd_gaming', 'd_clubbing', 'd_reading', 'd_tv', 'd_theater',
    'd_movies', 'd_concerts', 'd_music', 'd_shopping', 'd_yoga',
    'd_interests_correlate', 'd_expected_happy_with_sd_people',
    'd_expected_num_interested_in_me', 'd_expected_num_matches',
    'd_like', 'd_guess_prob_liked', 'expected_num_interested_in_me',
    'expected_num_matches', 'shared_interests_partner', 'shared_interests_o'
]

data = data.drop(columns=cols_to_drop, errors='ignore')

print(f"Number of remaining columns: {data.shape[1]}")

Number of remaining columns: 61


In [63]:
# List of columns to delete
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   gender                         8378 non-null   object
 1   age                            8283 non-null   object
 2   age_o                          8274 non-null   object
 3   d_d_age                        8378 non-null   object
 4   race                           8315 non-null   object
 5   race_o                         8305 non-null   object
 6   importance_same_race           8299 non-null   object
 7   importance_same_religion       8299 non-null   object
 8   field                          8315 non-null   object
 9   pref_o_attractive              8289 non-null   object
 10  pref_o_sincere                 8289 non-null   object
 11  pref_o_intelligence            8289 non-null   object
 12  pref_o_funny                   8280 non-null   object
 13  pre

In [64]:
## Nominal atributes
obj_cols = data.select_dtypes(include='object').columns

for col in obj_cols:
    print(col, data[col].unique()[:15])

gender ['female' 'male']
age ['21' '24' '25' '23' '22' '26' '27' '30' '28' <NA> '29' '34' '35' '32'
 '39']
age_o ['27' '22' '23' '24' '25' '30' '28' '21' '26' '29' <NA> '39' '32' '34'
 '35']
d_d_age ['[4-6]' '[0-1]' '[2-3]' '[7-37]']
race ["'Asian/Pacific Islander/Asian-American'" 'European/Caucasian-American'
 'Other' "'Latino/Hispanic American'" "'Black/African American'" <NA>]
race_o ['European/Caucasian-American' "'Asian/Pacific Islander/Asian-American'"
 "'Latino/Hispanic American'" 'Other' "'Black/African American'" <NA>]
importance_same_race ['2' '8' '1' '4' '7' '3' '9' '10' <NA> '5' '6' '0']
importance_same_religion ['4' '5' '1' '3' '2' '8' '10' '6' <NA> '7' '9']
field ['Law' 'law' 'Economics' "'Masters in Public Administration'"
 "'Masters of Social Work&Education'" 'Finance' 'Business'
 "'political science'" 'money' "'Operations Research'" "'TC [Health Ed]'"
 'Psychology' "'social work'" "'Social Work'"
 "'Speech Language Pathology'"]
pref_o_attractive ['35' '60' '19' '30' '5

In [65]:
data.replace('?', pd.NA, inplace=True)

In [66]:
## Nulls
data.shape[0] - data.dropna().shape[0]

1912

In [67]:
null_counts = data.isnull().sum()

null_counts_with_values = null_counts[null_counts != 0]

print(null_counts_with_values)

age                               95
age_o                            104
race                              63
race_o                            73
importance_same_race              79
importance_same_religion          79
field                             63
pref_o_attractive                 89
pref_o_sincere                    89
pref_o_intelligence               89
pref_o_funny                      98
pref_o_ambitious                 107
pref_o_shared_interests          129
attractive_o                     212
sinsere_o                        287
intelligence_o                   306
funny_o                          360
ambitous_o                       722
attractive_important              79
sincere_important                 79
intellicence_important            79
funny_important                   89
ambtition_important               99
shared_interests_important       121
attractive                       105
sincere                          105
intelligence                     105
f

In [12]:
data.describe()

,has_null,wave,d_age,samerace,decision,decision_o,match
count,8378.00000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000
mean,0.87491,11.350919,4.185605,0.395799,0.419909,0.419551,0.164717
std,0.33084,5.995903,4.596171,0.489051,0.493573,0.493515,0.370947
min,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.00000,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.00000,11.000000,3.000000,0.000000,0.000000,0.000000,0.000000
75%,1.00000,15.000000,5.000000,1.000000,1.000000,1.000000,0.000000
max,1.00000,21.000000,37.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
for col in obj_cols:
    print(col, data[col].unique()[:10])

,has_null,wave,d_age,samerace,decision,decision_o,match
count,8378.00000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000
mean,0.87491,11.350919,4.185605,0.395799,0.419909,0.419551,0.164717
std,0.33084,5.995903,4.596171,0.489051,0.493573,0.493515,0.370947
min,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.00000,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.00000,11.000000,3.000000,0.000000,0.000000,0.000000,0.000000
75%,1.00000,15.000000,5.000000,1.000000,1.000000,1.000000,0.000000
max,1.00000,21.000000,37.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
data.gender.value_counts()

gender
male      4194
female    4184
Name: count, dtype: int64

In [16]:
## Duplicates
duplicates = data.duplicated()
print(df[duplicates])

Empty DataFrame
Columns: [has_null, wave, gender, age, age_o, d_age, d_d_age, race, race_o, samerace, importance_same_race, importance_same_religion, d_importance_same_race, d_importance_same_religion, field, pref_o_attractive, pref_o_sincere, pref_o_intelligence, pref_o_funny, pref_o_ambitious, pref_o_shared_interests, d_pref_o_attractive, d_pref_o_sincere, d_pref_o_intelligence, d_pref_o_funny, d_pref_o_ambitious, d_pref_o_shared_interests, attractive_o, sinsere_o, intelligence_o, funny_o, ambitous_o, shared_interests_o, d_attractive_o, d_sinsere_o, d_intelligence_o, d_funny_o, d_ambitous_o, d_shared_interests_o, attractive_important, sincere_important, intellicence_important, funny_important, ambtition_important, shared_interests_important, d_attractive_important, d_sincere_important, d_intellicence_important, d_funny_important, d_ambtition_important, d_shared_interests_important, attractive, sincere, intelligence, funny, ambition, d_attractive, d_sincere, d_intelligence, d_funny, d

In [17]:
## Nulls
data.shape[0] - data.dropna().shape[0]

0